In [61]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def parse_reviews_from_html(html_content):
    # Se obtiene el codigo html y se accede a la parte de las reviews
    soup = BeautifulSoup(html_content, 'html.parser')
    review_elements = soup.find_all('div', class_='review-element')
    reviews_data = []
    
    for review in review_elements:
        # Se extrae el texto con una comprobacion
        review_text_element = review.find('div', class_='text')
        review_text = review_text_element.text.strip() if review_text_element else "No text found"

        # Se comprueba si la etiqueta de recomendación existe
        tags_div = review.find('div', class_='tags')
        
        if tags_div:
            # Se obtiene la polaridad de la reseña
            recommendation_status = tags_div.text.strip()
            
            # Se comprueba si la polaridad es una de las que queremos
            if recommendation_status in ["Recommended", "Not Recommended", "Mixed Feelings"]:
                # Se añade tanto el texto como la marca de polaridad
                reviews_data.append({
                    'review_text': review_text,
                    'polarity': recommendation_status
                })
            
    return reviews_data

def scrape_anime_reviews(anime_id, anime_title, max_reviews=100):  
    # Se inicializa la lista de reseñas y el parámetro de orden
    valid_reviews = []
    sort_by = 'suggested'
    base_url = f"https://myanimelist.net/anime/{anime_id}/{anime_title}/reviews?sort={sort_by}"
    print(f"Scrapeando reseñas desde: {base_url}")

    # Se usa un 'set' para guardar los textos que ya hemos visto
    seen_review_texts = set()

    # Se intenta obtener la primera página de reseñas
    try:
        response = requests.get(base_url, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()
        
        initial_reviews = parse_reviews_from_html(response.text)
        
        # Se añaden las reseñas de una en una, comprobando duplicados
        for review in initial_reviews:
            if review['review_text'] not in seen_review_texts:
                valid_reviews.append(review)
                seen_review_texts.add(review['review_text'])
        
    except requests.exceptions.RequestException as e:
        print(f"Error en la página inicial de reseñas: {e}")
        return [] 

    # Se prepara el contador para la paginación (Página 2 en adelante)
    page_number = 2
    
    while len(valid_reviews) < max_reviews:
        
        time.sleep(2) 
        load_more_url = f"{base_url}&p={page_number}"
        
        try:
            response = requests.get(load_more_url, headers={'User-Agent': 'Mozilla/5.0'})
            response.raise_for_status()
            
            additional_reviews = parse_reviews_from_html(response.text)
            
            # Si no hay más reseñas, se rompe el bucle
            if not additional_reviews:
                print("No hay más reseñas disponibles (filtro o fin real).")
                break 
            
            # Se comprueba si el texto de la reseña NO está en el set previamente creado
            new_reviews_found_count = 0
            for review in additional_reviews:
                if review['review_text'] not in seen_review_texts:
                    valid_reviews.append(review)
                    seen_review_texts.add(review['review_text'])
                    new_reviews_found_count += 1

            # Se comprueba que la pagina no sea una pagina repetida
            if new_reviews_found_count == 0 and additional_reviews:
                print("Página duplicada detectada. Deteniendo búsqueda.")
                break 

            page_number += 1
            
        except requests.exceptions.RequestException as e:
            print(f"Error en página {page_number}: {e}. Deteniendo.")
            break 
            
    # Se devuelve la lista final, cortada al máximo
    return valid_reviews[:max_reviews]

def get_top_anime_list(limit=100):
    # Se inicializan las variables para guardar la lista y manejar la paginación
    anime_list = []
    start_rank = 0
    print("Obteniendo la lista del top 100 de animes...")
    
    # Se inicia el bucle para scrapear páginas hasta alcanzar el límite
    while len(anime_list) < limit:
        
        # Se construye la URL correcta, manejando el caso especial de la página 1
        if start_rank == 0:
            url = "https://myanimelist.net/topanime.php"
        else:
            url = f"https://myanimelist.net/topanime.php?limit={start_rank}"

        print(f"Accediendo a: {url}")
        
        # Se intenta obtener la página de ranking
        try:
            response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error al acceder a la página de ranking: {e}")
            break

        # Se parsea el HTML y se buscan todas las filas de la tabla
        soup = BeautifulSoup(response.text, 'html.parser')
        ranking_rows = soup.find_all('tr', class_='ranking-list')
        
        # Se itera sobre cada fila (cada anime) encontrada en la página
        for row in ranking_rows:
            # Se busca el enlace principal que contiene la URL
            link_element = row.find('a', class_='hoverinfo_trigger')
            
            # Se comprueba si el enlace existe y si aún no hemos llegado al límite
            if link_element and len(anime_list) < limit:
                url_anime = link_element['href']
                
                # Se intenta extraer el ID y el 'slug' del título desde la URL
                try:
                    parts = url_anime.split('/')
                    anime_id = int(parts[4])
                    anime_title_slug = parts[5]
                    anime_list.append({'id': anime_id, 'title_slug': anime_title_slug})
                except (IndexError, ValueError):
                    continue
        
        # Se prepara para la siguiente página
        start_rank += 50
        time.sleep(1) # Pausa entre páginas del ranking

    print(f"Se encontraron {len(anime_list)} animes en el top.")
    return anime_list


if __name__ == "__main__":
    # Se definen las columnas que queremos al final
    reviews_complete_df = pd.DataFrame()
    
    # Se define el límite total de reseñas que queremos
    LIMITE_TOTAL = 10000
    
    # Se obtiene la lista de los 150 peimweoa animes para tener margen
    top_animes = get_top_anime_list(limit=150) 
    
    print(f"Iniciando scrapeo. Objetivo: {LIMITE_TOTAL} reseñas.")
    
    # Se comprueba si la lista de animes se obtuvo correctamente
    if not top_animes:
        print("No se pudo obtener la lista de animes. Saliendo del programa.")
    else:
        # Se itera sobre cada anime de la lista para procesarlo
        for i, anime in enumerate(top_animes):
            
            print(f"Procesando Anime {i+1}/{len(top_animes)}: {anime['title_slug']}")
            
            # Se obtienen las reseñas para el anime actual
            reseñas = scrape_anime_reviews(anime['id'], anime['title_slug'], max_reviews=100)
            
            # Se comprueba si se obtuvieron reseñas
            if reseñas:
                # Se convierten las reseñas (lista) a un DataFrame temporal
                temp_df = pd.DataFrame(reseñas)
                
                # Se añade la columna 'tittle' usando el 'slug' del anime
                temp_df['tittle'] = anime['title_slug']
                
                # Se añade este DataFrame temporal al DataFrame global
                reviews_complete_df = pd.concat([reviews_complete_df, temp_df], ignore_index=True)
                
                print(f"Añadidas {len(reseñas)} reseñas. Total actual: {len(reviews_complete_df)}")
            else:
                print(f"No se encontraron reseñas para {anime['title_slug']}")
            
            # Se comprueba si hemos alcanzado el límite total
            if len(reviews_complete_df) >= LIMITE_TOTAL:
                print(f"Límite de {LIMITE_TOTAL} reseñas alcanzado.")
                break 
            
            # Se aplica una pausa de cortesía entre cada anime
            time.sleep(3)
    
    # Se muestra la cantidad de reseñas guardadas totales.
    print(f"Guardando {len(reviews_complete_df)} reseñas totales en 'reviews_complete.csv'.")
    
    # Se reordenan las columnas para dejar la columna objetivo la última
    reviews_complete_df = reviews_complete_df[['tittle', 'review_text', 'polarity']]
    
    # Se guarda el DataFrame en un único archivo CSV
    reviews_complete_df.to_csv("reviews_complete.csv", index=False, encoding='utf-8-sig')
    
    print("Proceso de scraping masivo completado")

Obteniendo la lista del top 100 de animes...
Accediendo a: https://myanimelist.net/topanime.php
Accediendo a: https://myanimelist.net/topanime.php?limit=50
Accediendo a: https://myanimelist.net/topanime.php?limit=100
Se encontraron 150 animes en el top.
Iniciando scrapeo. Objetivo: 10000 reseñas.
Procesando Anime 1/150: Sousou_no_Frieren
Scrapeando reseñas desde: https://myanimelist.net/anime/52991/Sousou_no_Frieren/reviews?sort=suggested
No hay más reseñas disponibles (filtro o fin real).
Añadidas 13 reseñas. Total actual: 13
Procesando Anime 2/150: Chainsaw_Man_Movie__Reze-hen
Scrapeando reseñas desde: https://myanimelist.net/anime/57555/Chainsaw_Man_Movie__Reze-hen/reviews?sort=suggested
Añadidas 100 reseñas. Total actual: 113
Procesando Anime 3/150: Fullmetal_Alchemist__Brotherhood
Scrapeando reseñas desde: https://myanimelist.net/anime/5114/Fullmetal_Alchemist__Brotherhood/reviews?sort=suggested
Añadidas 100 reseñas. Total actual: 213
Procesando Anime 4/150: Steins_Gate
Scrapeando